In [1]:
import copy
import itertools

class PuzzleNode:
    """
    Class for 8-puzzle and its generalizations
    Attributes (in corresponding order):
    - State of the puzzle
    - Board size n
    - g-value: the cost to reach the node
    - h-value: the cost to reach the goal from this node
    - f-value: total cost for this node
    - Pointer to the parent node (default None)
    - Flag whether to prune the node
    """
    def __init__(self, state, gval, hval, parent=None):
        """Initialize the node"""
        self.state = state
        self._n = len(state)
        self.gval = gval 
        self.hval = hval
        self.fval = self._f_cost()
        self.parent = parent
        self.pruned = False

    def __str__(self):
        """Print out the grid showing the board state"""
        st = ''
        for row in self.state:
            st += ' '.join(map(str, row))
            st += '\r\n'
        return st
    
    def __lt__(self, other):
        """Define 'less than' comparison between two nodes"""
        return self.fval < other.fval
    
    def _check_parent(self, child):
        """Check if a move leads back to the parent"""
        if self.parent is not None:
            if self.parent.state == child:
                return True
        return False
    
    def _f_cost(self):
        """Calculate the total cost for the node"""
        return self.gval + self.hval
    
    def generate_children(self, heuristic):
        """Generate child nodes from legal moves"""
        lst = range(self._n)
        loops = list(itertools.product(lst, lst))
        # Find the position of the blank tile
        for row, col in loops:
            if self.state[row][col] == 0:
                break
        # Check legal move,
        # then copy over state and swap the two tiles
        # lastly, if new state does not revert back to the parent,
        # create a new node.
        
        # If the blank tile is not in the right-most column,
        # then we can slide a tile to the right
        if col < self._n-1:
            child = copy.deepcopy(self.state)
            child[row][col], child[row][col+1] = \
                child[row][col+1], child[row][col]
            if not self._check_parent(child):
                yield PuzzleNode(child, self.gval+1, 
                                 heuristic(child), parent = self)
        
        # If the blank tile is not in the left-most column,
        # then we can slide a tile to the left
        if col > 0:
            child = copy.deepcopy(self.state)
            child[row][col], child[row][col-1] = \
                child[row][col-1], child[row][col]
            if not self._check_parent(child):
                yield PuzzleNode(child, self.gval+1,
                                heuristic(child), parent = self)

        # If the blank tile is not in the bottom row,
        # then we can slide a tile down
        if row < self._n-1:
            child = copy.deepcopy(self.state)
            child[row][col], child[row+1][col] = \
                child[row+1][col], child[row][col]
            if not self._check_parent(child):
                yield PuzzleNode(child, self.gval+1,
                                heuristic(child), parent = self)

        # If the blank tile is not in the top row,
        # then we can slide a tile up
        if row > 0:
            child = copy.deepcopy(self.state)
            child[row][col], child[row-1][col] = \
                child[row-1][col], child[row][col]
            if not self._check_parent(child):
                yield PuzzleNode(child, self.gval+1,
                                heuristic(child), parent = self)

---

In [2]:
class Memoize:
    """Memoization class"""
    def __init__(self, f):
        self._f = f
        self.memo = {}
    
    def __call__(self, *args):
        hash_val = str(args)
        # If not in memo yet, then add to memo
        if hash_val not in self.memo:
            self.memo[hash_val] = self._f(*args)
        # Return value stored in memo
        return self.memo[hash_val]

In [3]:
from collections import deque
class PatternDB:
    """Class for the pattern database"""
    def __init__(self, pattern):
        self.pattern = pattern # Store the pattern
        self.db = {} # Store the database
        
    # Can also create another class for PatternState
    # that inherits from the PuzzleNode class
    # and then overwrite the generate_children method
    # but this is easier to implement.
    # This function is actually just a placeholder since
    # we use breadth-first search using deque here.
    def heuristic(self, state):
        """Make search behave like uniform-cost search"""
        return 0
    
    def add(self, state, steps):
        """Add state and value to database"""
        key = mask(state.state, self.pattern)
        if key not in self.db:
            self.db[key] = steps

@Memoize
def mask(state, pattern):
    """Replace items not in the pattern with a dummy value"""
    n = len(state)
    masked_state = copy.deepcopy(state)
    loops = list(itertools.product(range(n), range(n)))
    for row, col in loops:            
        if masked_state[row][col] not in pattern:
            if masked_state[row][col] != 0:
                masked_state[row][col] = -1
    return str(masked_state)            
    
@Memoize
def get_goal(n):
    """Generate the goal state for the board size from input"""
    goal = []
    numbers = range(n**2)
    for i in range(n):
        goal.append(list(numbers[i*n:(i+1)*n]))
    return goal
    
def train_pattern(goal, pattern):
    """Construct the database object"""
    # Initialize the database
    db = PatternDB(pattern)
    # Create the goal node
    # As the start node is the goal state, the g-value now
    # represents the step taken to get to the goal
    goal = PuzzleNode(goal, 0, 0)
    frontier = deque()
    frontier.append(goal)
    explored_cost = {}
    # Perform breadth-first search
    while frontier:
        cur_state = frontier.popleft()
        if cur_state.pruned:
            continue
        # Add entry to database
        db.add(cur_state, cur_state.gval)
        # Generate new nodes
        for child in cur_state.generate_children(db.heuristic):
            state = mask(child.state, pattern)
            # Since we do not want our heuristic to overestimate,
            # we choose the lower gval
            if str(state) in explored_cost:
                if explored_cost[str(state)].gval > child.gval:
                    explored_cost[str(child.state)].pruned = True
                else:
                    continue
            frontier.append(child)
            explored_cost[str(state)] = child
    return db

def fringe_pattern(n):
    """Returns value for the fringe pattern"""
    fringe = []
    for i in range(n**2):
        if i%n == n-1 or i//n == n-1:
            fringe.append(i)
    return fringe

def half_pattern(n):
    """Returns value for the 'half pattern'"""
    # Here, the 'half pattern' can be seen as a generalization
    # of the 7-8 partition for the 15-puzzle
    return range(n**2//2)

goal_3 = get_goal(3)
db1 = train_pattern(goal_3, fringe_pattern(3))
db2 = train_pattern(goal_3, half_pattern(3))

In [4]:
import numpy as np
from itertools import chain

# Question 3.1
def validArrangement(n, state):
    """Check if a configuration is valid"""
    # Check the dimensions
    if np.shape(state) != (n, n):
        return False
    
    # Check the numbers
    check_list = set(range(n**2))
    cur_state = list(chain.from_iterable(state))
    if set(cur_state) != check_list:
        return False
    return True

# Extension 1: Check if a configuration can be solved
def inversionCount(state):
    """Count the number of inversions in a state"""
    count = 0
    for i in range(len(state)):
        if state[i] == 0:
            continue
        for j in range(i, len(state)):
            if state[i] > state[j] and state[j] != 0:
                count += 1
    return count

def solvableArrangement(n, state):
    """Check if a configuration can be solved"""
    cur_state = list(chain.from_iterable(state))
    # If the board size is odd, then it only depends on 
    # the number of inversions
    if n%2 == 1:
        score = inversionCount(cur_state)
    # If the board size is even, then it also depends on 
    # the row that has the blank tile
    else: 
        score = inversionCount(cur_state)+(cur_state.index(0)//n)
    if score %2 == 1:
        return False
    return True

@Memoize   
def h1(state):
    """Number of misplaced tiles heuristic"""
    n = len(state)
    misplaced = 0
    cur_state = list(chain.from_iterable(state))
    for i in range(n**2):
        if cur_state[i] != i:
            misplaced += 1
    return misplaced

@Memoize
def h2(state):
    """Manhattan distance heuristic"""
    n = len(state)
    distance = 0   
    loops = list(itertools.product(range(n), range(n)))
    for row, col in loops:
        if state[row][col] < 0:
            continue
        x_diff = abs(row - state[row][col]//n)
        y_diff = abs(col - state[row][col]%n)
        distance += x_diff + y_diff
    return distance

@Memoize
def h3(state):
    """Pattern database heuristic"""
    n = len(state)
    if n != 3:
        raise ValueError("Only has database for board size of 3.")
    pattern_1 = db1.db[mask(state,db1.pattern)]
    pattern_2 = db2.db[mask(state,db2.pattern)]
    return max(pattern_1, pattern_2)

heuristics = [h1, h2, h3]

In [5]:
from queue import PriorityQueue

def solvePuzzle(n, state, heuristic, prnt=False):
    """Puzzle solving algorithm"""
    # Check if board size is legal
    if n >= 128 or n < 2 :
        raise ValueError("Board size must be between 2 and 128.")
    # Check if configuration is valid
    if not validArrangement(n, state):
        return 0, 0, -1
    # Check if configuration is solvable
    if not solvableArrangement(n, state):
        return 0, 0, -2
    # Generate goal state
    goal = get_goal(n)
    # Create current node
    current_state = PuzzleNode(state, 0, heuristic(state))
    # Check if current state is goal state
    if state == goal:
        if prnt:
            print(current_state)
        return 0, 0, 0
    frontier = PriorityQueue() 
    frontier.put(current_state)
    frontier_size = 1
    explored_cost = {}
    # Perform A-star search
    while not frontier.empty():
        current_state = frontier.get()
        # If state is marked for pruning, then skip to next one
        if current_state.pruned:
            continue
        # Generate new nodes
        for child in current_state.generate_children(heuristic):
            # Check if the new node is the goal state
            if child.state == goal:
                # Trace back and print out path
                if prnt:
                    current = child
                    path = [current]
                    while current.parent is not None:
                        path.append(current.parent)
                        current = current.parent
                    for node in path[::-1]:
                        print(node)
                return child.gval, frontier_size, 0
            
            if str(child.state) in explored_cost:
                if explored_cost[str(child.state)].gval > child.gval:
                    # Mark old node for pruning
                    explored_cost[str(child.state)].pruned = True
                else:
                    continue
            # Else, add child to frontier if the state has not been
            # explored or is has a lower g-value
            frontier.put(child)
            if frontier_size < frontier.qsize():
                frontier_size = frontier.qsize()
            explored_cost[str(child.state)] = child

---

In [6]:
from datetime import datetime
def test_code(state):
    n = len(state)
    steps = []
    frontier_size = []
    time = []
    for i in heuristics:
        start = datetime.now()
        step, frontier, error = (solvePuzzle(n, state, i))
        end = datetime.now()
        steps.append(step)
        frontier_size.append(frontier)
        time.append((end-start).total_seconds())
    return steps, frontier_size, time

In [7]:
import pandas
test_1 = [[5,7,6],[2,4,3],[8,1,0]]
print('Test case 1: ', test_1)
steps, frontier_size, time = test_code(test_1)

data = [steps, frontier_size, time]
heuristic_list = ["Misplaced tiles", "Manhattan distance",
                  "Pattern database"]
headers = ["Steps", "Frontier size", "Time (s)"]
pandas.DataFrame(data, headers, heuristic_list)

Test case 1:  [[5, 7, 6], [2, 4, 3], [8, 1, 0]]


,Misplaced tiles,Manhattan distance,Pattern database
Steps,28.000000,28.00000,28.000000
Frontier size,20756.000000,2529.00000,101.000000
Time (s),5.521223,0.39753,0.017981


In [8]:
test_2 = [[7,0,8],[4,6,1],[5,3,2]]
print('Test case 2: ', test_2)
steps, frontier_size, time = test_code(test_2)

data = [steps, frontier_size, time]
pandas.DataFrame(data, headers, heuristic_list)

Test case 2:  [[7, 0, 8], [4, 6, 1], [5, 3, 2]]


,Misplaced tiles,Manhattan distance,Pattern database
Steps,25.000000,25.000000,25.000000
Frontier size,11912.000000,1918.000000,42.000000
Time (s),2.373447,0.331726,0.007449


In [9]:
test_3 = [[2,3,7],[1,8,0],[6,5,4]]
print('Test case 3: ', test_3)
steps, frontier_size, time = test_code(test_3)

data = [steps, frontier_size, time]
pandas.DataFrame(data, headers, heuristic_list)

Test case 3:  [[2, 3, 7], [1, 8, 0], [6, 5, 4]]


,Misplaced tiles,Manhattan distance,Pattern database
Steps,17.00000,17.000000,17.000000
Frontier size,451.00000,121.000000,18.000000
Time (s),0.06705,0.022316,0.003349


In [10]:
test_4 = [[0,10,14,12], [13,6,15,8], [9,2,11,4], [5,1,7,3]]
print('Test case 4: ', test_4)
steps = []
frontier_size = []
time = []
start = datetime.now()
step, frontier, error = (solvePuzzle(4, test_4, heuristics[1]))
end = datetime.now()
steps.append(step)
frontier_size.append(frontier)
time.append((end-start).total_seconds())

heuristic_list = ["Manhattan distance"]
headers = ["Steps", "Frontier size", "Time (s)"]
data = [steps, frontier_size, time]
pandas.DataFrame(data, headers, heuristic_list)

Test case 4:  [[0, 10, 14, 12], [13, 6, 15, 8], [9, 2, 11, 4], [5, 1, 7, 3]]


,Manhattan distance
Steps,56.000000
Frontier size,240908.000000
Time (s),44.482121
